## 1. 예측할 레이블 (평가 지표) 정의하기

##### 우선 무엇을 예측할지를 정해야 합니다.
예시로, “상담 결과”(해결 불가 / 만족 / 미흡 / 추가 상담 필요)를 레이블로 삼아보겠습니다.

팁: 이 값은 이미 여러분이 만든 merged_classification_*_final.json 안의
instructions 블록 중, task_category == "상담 결과" 의 output 필드에 들어 있습니다.

##### 1-1) 분류 JSON에서 상담 결과만 뽑아서 DataFrame으로 만들기

In [3]:
import glob, json
import pandas as pd
from tqdm import tqdm

# classification 결과 JSON 폴더
CLASS_DIR = 'json_merge/classification_merge_output_v3'  
# 예: output_classification/merged_classification_40001_final.json 등

rows = []
for fp in tqdm(glob.glob(f'{CLASS_DIR}/merged_classification_*_final.json'), desc="Loading labels"):
    rec = json.load(open(fp, 'r', encoding='utf-8'))
    sid = rec['session_id']
    # 분류 리스트에서 첫 아이템만 보자 (세션당 동일 상담 내용)
    block = rec['분류'][0]
    # 그 block 안의 instructions[0]['data'] 에서 '상담 결과'를 찾기
    label = None
    for d in block['instructions'][0]['data']:
        if d['task_category'] == '상담 결과':
            label = d['output']
            break
    rows.append({'session_id': sid, 'result_label': label})

df_labels = pd.DataFrame(rows)
print(df_labels['result_label'].value_counts())

Loading labels: 100%|████████████████████| 3600/3600 [00:00<00:00, 21609.02it/s]

result_label
만족          2982
추가 상담 필요     398
미흡           154
해결 불가         66
Name: count, dtype: int64


In [5]:
df_labels.tail()

,session_id,result_label
3595,40398,만족
3596,43241,만족
3597,41815,만족
3598,40687,만족
3599,42542,해결 불가


In [5]:
import os

# 1번에서 생성된 df_labels DataFrame 을 CSV로 저장하기
os.makedirs('coloums_extraction/preprocessing', exist_ok=True)
df_labels.to_csv('coloums_extraction/preprocessing/session_labels.csv', index=False, encoding='utf-8-sig')

print("✅ 세션별 레이블 CSV 저장 완료 → dataset/preprocessing/session_labels.csv")

✅ 세션별 레이블 CSV 저장 완료 → dataset/preprocessing/session_labels.csv


## 2. 특성 csv와 레이블 합치기 (JOIN)

이제 1단계에서 만든 레이블(df_labels)을, 이전에 뽑아둔 텍스트·메타 특성 CSV(text_features_all.csv)와 session_id 기준으로 합치겠습니다.

In [9]:
import pandas as pd

# 1) 불러오기
df_feat   = pd.read_csv('coloumns_extracion/text_features_all_v3.csv', encoding='utf-8-sig')
df_labels = pd.read_csv('dataset/preprocessing/session_labels.csv', encoding='utf-8-sig')  # 혹은 위에서 만든 df_labels.to_csv(...) 파일

# 2) session_id 자료형 통일 (둘 다 str으로)
df_feat['session_id']   = df_feat['session_id'].astype(str)
df_labels['session_id'] = df_labels['session_id'].astype(str)

# 3) 병합
df = df_feat.merge(df_labels, on='session_id', how='inner')

# 4) 확인
print(f"결측 레이블: {df['result_label'].isna().sum()}개")
print(f"총 샘플: {len(df)}개, 레이블 분포:\n", df['result_label'].value_counts())

결측 레이블: 0개
총 샘플: 3533개, 레이블 분포:
 result_label
만족          2926
추가 상담 필요     390
미흡           151
해결 불가         66
Name: count, dtype: int64


In [11]:
df.head()

,session_id,speech_count,asr_segments,top_nouns,고객_emo_1_star_score,고객_emo_2_star_score,고객_emo_3_star_score,고객_emo_4_star_score,고객_emo_5_star_score,고객_sent_score,...,positive_word_ratio,euphonious_word_ratio,confirmation_ratio,empathy_ratio,apology_ratio,request_ratio,alternative_suggestion_count,conflict_flag,manual_compliance_ratio,result_label
0,40001,49,"[{'speaker': '상담사', 'text': '일상에 즐거운 변화 U+ <NA...","거,네,고객,이제,매장,코드,그,카드,제,그거",0.236101,0.213667,0.238402,0.149915,0.161916,2.787877,...,0.006250,0.012500,0.001563,0.0,0.00625,0.001563,0,0,1,해결 불가
1,40002,62,"[{'speaker': '상담사', 'text': '일상에 즐거운 변화 LGU+ <...","네,예,방문,고객,명의,확인,번호,그,지금,이",0.188101,0.220787,0.298281,0.162566,0.130265,2.826106,...,0.010327,0.001721,0.015491,0.0,0.00000,0.000000,0,1,1,만족
2,40003,77,"[{'speaker': '상담사', 'text': '일상에 즐거운 변화 LGU+ <...","네,금제,할인,변경,거,수,요금,제,부터,고객",0.262344,0.257603,0.247805,0.111836,0.120412,2.570370,...,0.006309,0.004732,0.003155,0.0,0.00000,0.001577,0,0,1,만족
3,40004,52,"[{'speaker': '상담사', 'text': '일상에 즐거운 변화 LGU+ <...","네,이제,고객,확인,그,제,저희,거,안,뭐",0.262972,0.205973,0.218362,0.144251,0.168442,2.749217,...,0.001527,0.001527,0.021374,0.0,0.00000,0.000000,0,0,1,미흡
4,40005,63,"[{'speaker': '상담사', 'text': '일상에 즐거운 변화 LGU+ <...","네,고객,통화,거,지금,할인,중,대기,서비스,번",0.179392,0.170757,0.229438,0.192581,0.227833,3.118704,...,0.003436,0.006873,0.006873,0.0,0.00000,0.001718,0,0,1,만족


## 3. 학습/검증/시험용 데이터 분할

##### result_label 의 분포 비율(클래스 불균형)을 유지하며 Stratified Split 합니다.

In [13]:
from sklearn.model_selection import train_test_split

# 3-1) Test 세트 분리 (15%)
trainval, test = train_test_split(
    df,
    test_size=0.15,
    stratify=df['result_label'],
    random_state=42
)

# 3-2) Train / Validation 분리 (전체 중 15%가 Validation → trainval 의 17.6% 약 15%)
train, val = train_test_split(
    trainval,
    test_size=0.1765,
    stratify=trainval['result_label'],
    random_state=42
)

print("train:", train.shape, "val:", val.shape, "test:", test.shape)

train: (2472, 33) val: (531, 33) test: (530, 33)


## 4. 피처 선택 및 전처리
1. 불필요 컬럼 제거
- asr_segments 같은 리스트/텍스트 컬럼은 모델 입력용으로 가공하거나 제거
- top_nouns 는 나중에 TF-IDF → embedding 용으로 따로 저장 가능

2. 인코딩
- 범주형 컬럼(mid_category, content_category, rec_place, result_label) → LabelEncoder or OneHot

3. 스케일링
- 수치형 (speech_count, 비율 컬럼 등) → StandardScaler 또는 MinMaxScaler

In [15]:
# 예시: 범주형 인코딩
from sklearn.preprocessing import LabelEncoder

le_topic = LabelEncoder().fit(df['mid_category'])
train['mid_category_id'] = le_topic.transform(train['mid_category'])
val['mid_category_id']   = le_topic.transform(val['mid_category'])
test['mid_category_id']  = le_topic.transform(test['mid_category'])

# 결과 레이블도 숫자로
le_res = LabelEncoder().fit(df['result_label'])
train['label_id'] = le_res.transform(train['result_label'])
val['label_id']   = le_res.transform(val['result_label'])
test['label_id']  = le_res.transform(test['result_label'])

## 5. 최종 데이터셋 csv로 저장

In [17]:
os.makedirs('dataset_v3', exist_ok=True)
train.to_csv('dataset_v3/train.csv', index=False, encoding='utf-8-sig')
val.to_csv  ('dataset_v3/val.csv',   index=False, encoding='utf-8-sig')
test.to_csv ('dataset_v3/test.csv',  index=False, encoding='utf-8-sig')
print("✅ Dataset saved under dataset_v3/") 

✅ Dataset saved under dataset_v3/
